# 5003 TITLE

* 可以用schema初始化数据类型，但是还需要对应位置，就神烦，不整了，GTMD
* 如果用这个方法，需要在读取数据的时候增加参数schema，这里设置的结构就叫schema，所以直接写schema=schema
* 还需要导入包：`from pyspark.sql.types import *`
* schema写法：`schema = StructType([StructField("id", IntegerType()),StructField("title", StringType()),StructField("tagline", StringType()),StructField("vote_average", FloatType()),StructField("vote_count", IntegerType())])`
* 当你读取csv格式错误的时候，你就加一个`multiLine=True,escape='"'`，专治各种不服

## UDF是解决DataFrame问题的通法😂

In [1]:
# 导入各种用到的包
# 返回值类型支持
from pyspark.sql.types import ArrayType
from pyspark.sql.types import MapType
from pyspark.sql.types import StringType
from pyspark.sql.types import NullType
# UDF模块
from pyspark.sql.functions import udf

* 形如`[{'id': 16,...},{????},]`
* 这次处理的结构大致是list里面含有maps，但是整体的类型又是string。

In [28]:
# 类型转换：
typetransfer = udf(lambda x:eval(x), ArrayType(MapType(StringType(),StringType())))

In [29]:
# 把genres提取出来，再建立一个UDF来实现这个功能。
takegenres = udf(lambda x: [i['name'] for i in x], StringType())

In [31]:
# 去掉外面的中括号，这是上面的代码处理list时候的衍生物
rmbracket = udf(lambda x:','.join(x), StringType())

In [ ]:
bmd4 = bmd3.select('id',takegenres('genres').alias('genres'),'tagline','title','vote_average','vote_count')

# 处理movies_metadata.csv

In [5]:
df1 = spark.read.csv('/Users/apple/Desktop/5003-project/moviedata/TMD/movies_metadata.csv',multiLine=True,escape='"', header=True, inferSchema=True)

In [6]:
# 一眼就能看出来这个具体结构一眼看不出来，那就不看了吧
df1.show()

+-----+---------------------+--------+--------------------+--------------------+-----+---------+-----------------+--------------------+--------------------+----------+--------------------+--------------------+--------------------+------------+---------+-------+--------------------+--------+--------------------+--------------------+-----+------------+----------+
|adult|belongs_to_collection|  budget|              genres|            homepage|   id|  imdb_id|original_language|      original_title|            overview|popularity|         poster_path|production_companies|production_countries|release_date|  revenue|runtime|    spoken_languages|  status|             tagline|               title|video|vote_average|vote_count|
+-----+---------------------+--------+--------------------+--------------------+-----+---------+-----------------+--------------------+--------------------+----------+--------------------+--------------------+--------------------+------------+---------+-------+-----------

In [13]:
# 看一看这个数据的结构和类型，以及包含哪些特征
print(df1)
print('------------------分界线------------------')
df1.printSchema()
# 有的数据类型并不如你所愿，记得转换

DataFrame[adult: string, belongs_to_collection: string, budget: string, genres: string, homepage: string, id: string, imdb_id: string, original_language: string, original_title: string, overview: string, popularity: string, poster_path: string, production_companies: string, production_countries: string, release_date: string, revenue: bigint, runtime: double, spoken_languages: string, status: string, tagline: string, title: string, video: boolean, vote_average: double, vote_count: int]
------------------分界线------------------
root
 |-- adult: string (nullable = true)
 |-- belongs_to_collection: string (nullable = true)
 |-- budget: string (nullable = true)
 |-- genres: string (nullable = true)
 |-- homepage: string (nullable = true)
 |-- id: string (nullable = true)
 |-- imdb_id: string (nullable = true)
 |-- original_language: string (nullable = true)
 |-- original_title: string (nullable = true)
 |-- overview: string (nullable = true)
 |-- popularity: string (nullable = true)
 |-- post

In [17]:
# 抽出我们需要的那些特征,让他们看上去更简洁
bmd = df1.select('genres','id','tagline','title','vote_average','vote_count')
bmd

DataFrame[genres: string, id: string, tagline: string, title: string, vote_average: double, vote_count: int]

In [19]:
# 把id的数据类型转换成int，并放在前面
bmd2 = bmd.select(bmd['id'].astype('int'),'genres','tagline','title','vote_average','vote_count')
bmd2

DataFrame[id: int, genres: string, tagline: string, title: string, vote_average: double, vote_count: int]

In [21]:
# 这里bmd2中genres一列的内容很乱，需要单独处理，取出关键词。
# 使用前面提到的UDF函数来解决这个问题。
bmd2.show()

+-----+--------------------+--------------------+--------------------+------------+----------+
|   id|              genres|             tagline|               title|vote_average|vote_count|
+-----+--------------------+--------------------+--------------------+------------+----------+
|  862|[{'id': 16, 'name...|                null|           Toy Story|         7.7|      5415|
| 8844|[{'id': 12, 'name...|Roll the dice and...|             Jumanji|         6.9|      2413|
|15602|[{'id': 10749, 'n...|Still Yelling. St...|    Grumpier Old Men|         6.5|        92|
|31357|[{'id': 35, 'name...|Friends are the p...|   Waiting to Exhale|         6.1|        34|
|11862|[{'id': 35, 'name...|Just When His Wor...|Father of the Bri...|         5.7|       173|
|  949|[{'id': 28, 'name...|A Los Angeles Cri...|                Heat|         7.7|      1886|
|11860|[{'id': 35, 'name...|You are cordially...|             Sabrina|         6.2|       141|
|45325|[{'id': 28, 'name...|The Original Bad ...| 

In [32]:
# 先转换成类型，方便之后的操作
bmd3 = bmd2.select('id',typetransfer('genres').alias('genres'),'tagline','title','vote_average','vote_count')
# 取出关键词
bmd4 = bmd3.select('id',takegenres('genres').alias('genres'),'tagline','title','vote_average','vote_count')
bmd5 = bmd4.select('id',rmbracket('genres').alias('genres'),'tagline','title','vote_average','vote_count')
bmd5.show()

+-----+--------------------+--------------------+--------------------+------------+----------+
|   id|              genres|             tagline|               title|vote_average|vote_count|
+-----+--------------------+--------------------+--------------------+------------+----------+
|  862|Animation,Comedy,...|                null|           Toy Story|         7.7|      5415|
| 8844|Adventure,Fantasy...|Roll the dice and...|             Jumanji|         6.9|      2413|
|15602|      Romance,Comedy|Still Yelling. St...|    Grumpier Old Men|         6.5|        92|
|31357|Comedy,Drama,Romance|Friends are the p...|   Waiting to Exhale|         6.1|        34|
|11862|              Comedy|Just When His Wor...|Father of the Bri...|         5.7|       173|
|  949|Action,Crime,Dram...|A Los Angeles Cri...|                Heat|         7.7|      1886|
|11860|      Comedy,Romance|You are cordially...|             Sabrina|         6.2|       141|
|45325|Action,Adventure,...|The Original Bad ...| 

In [33]:
# 再单独取出一些数据
bmd6 = bmd5.select('id','genres','vote_average','vote_count')
bmd6.show()

+-----+--------------------+------------+----------+
|   id|              genres|vote_average|vote_count|
+-----+--------------------+------------+----------+
|  862|Animation,Comedy,...|         7.7|      5415|
| 8844|Adventure,Fantasy...|         6.9|      2413|
|15602|      Romance,Comedy|         6.5|        92|
|31357|Comedy,Drama,Romance|         6.1|        34|
|11862|              Comedy|         5.7|       173|
|  949|Action,Crime,Dram...|         7.7|      1886|
|11860|      Comedy,Romance|         6.2|       141|
|45325|Action,Adventure,...|         5.4|        45|
| 9091|Action,Adventure,...|         5.5|       174|
|  710|Adventure,Action,...|         6.6|      1194|
| 9087|Comedy,Drama,Romance|         6.5|       199|
|12110|       Comedy,Horror|         5.7|       210|
|21032|Family,Animation,...|         7.1|       423|
|10858|       History,Drama|         7.1|        72|
| 1408|    Action,Adventure|         5.7|       137|
|  524|         Drama,Crime|         7.8|     

In [38]:
# 把genres提取出来，再建立一个UDF来实现这个功能。
takegenres = udf(lambda x: [i['name'] for i in x], StringType())
bmd4 = bmd3.select('id',takegenres('genres').alias('genres'),'tagline','title','vote_average','vote_count')

In [42]:
bmd4.show()

+-----+--------------------+--------------------+--------------------+------------+----------+
|   id|              genres|             tagline|               title|vote_average|vote_count|
+-----+--------------------+--------------------+--------------------+------------+----------+
|  862|[Animation, Comed...|                null|           Toy Story|         7.7|      5415|
| 8844|[Adventure, Fanta...|Roll the dice and...|             Jumanji|         6.9|      2413|
|15602|   [Romance, Comedy]|Still Yelling. St...|    Grumpier Old Men|         6.5|        92|
|31357|[Comedy, Drama, R...|Friends are the p...|   Waiting to Exhale|         6.1|        34|
|11862|            [Comedy]|Just When His Wor...|Father of the Bri...|         5.7|       173|
|  949|[Action, Crime, D...|A Los Angeles Cri...|                Heat|         7.7|      1886|
|11860|   [Comedy, Romance]|You are cordially...|             Sabrina|         6.2|       141|
|45325|[Action, Adventur...|The Original Bad ...| 

In [15]:
bmd4.dtypes

[('id', 'int'), ('genres', 'string'), ('tagline', 'string'), ('title', 'string'), ('vote_average', 'double'), ('vote_count', 'int')]

In [52]:
bmd5 = bmd4.select('id',typetransfer2str('genres').alias('genres'),'vote_average','vote_count')
bmd5.show()

+-----+--------------------+------------+----------+
|   id|              genres|vote_average|vote_count|
+-----+--------------------+------------+----------+
|  862|Animation,Comedy,...|         7.7|      5415|
| 8844|Adventure,Fantasy...|         6.9|      2413|
|15602|      Romance,Comedy|         6.5|        92|
|31357|Comedy,Drama,Romance|         6.1|        34|
|11862|              Comedy|         5.7|       173|
|  949|Action,Crime,Dram...|         7.7|      1886|
|11860|      Comedy,Romance|         6.2|       141|
|45325|Action,Adventure,...|         5.4|        45|
| 9091|Action,Adventure,...|         5.5|       174|
|  710|Adventure,Action,...|         6.6|      1194|
| 9087|Comedy,Drama,Romance|         6.5|       199|
|12110|       Comedy,Horror|         5.7|       210|
|21032|Family,Animation,...|         7.1|       423|
|10858|       History,Drama|         7.1|        72|
| 1408|    Action,Adventure|         5.7|       137|
|  524|         Drama,Crime|         7.8|     

_______________________________________________________________

In [17]:
cres = spark.read.csv('/Users/apple/Desktop/5003-project/moviedata/TMD/credits.csv',multiLine=True,escape='"', header=True, inferSchema=True)

In [18]:
cres.show()

+--------------------+--------------------+-----+
|                cast|                crew|   id|
+--------------------+--------------------+-----+
|[{'cast_id': 14, ...|[{'credit_id': '5...|  862|
|[{'cast_id': 1, '...|[{'credit_id': '5...| 8844|
|[{'cast_id': 2, '...|[{'credit_id': '5...|15602|
|[{'cast_id': 1, '...|[{'credit_id': '5...|31357|
|[{'cast_id': 1, '...|[{'credit_id': '5...|11862|
|[{'cast_id': 25, ...|[{'credit_id': '5...|  949|
|[{'cast_id': 1, '...|[{'credit_id': '5...|11860|
|[{'cast_id': 2, '...|[{'credit_id': '5...|45325|
|[{'cast_id': 1, '...|[{'credit_id': '5...| 9091|
|[{'cast_id': 1, '...|[{'credit_id': '5...|  710|
|[{'cast_id': 1, '...|[{'credit_id': '5...| 9087|
|[{'cast_id': 9, '...|[{'credit_id': '5...|12110|
|[{'cast_id': 1, '...|[{'credit_id': '5...|21032|
|[{'cast_id': 1, '...|[{'credit_id': '5...|10858|
|[{'cast_id': 1, '...|[{'credit_id': '5...| 1408|
|[{'cast_id': 4, '...|[{'credit_id': '5...|  524|
|[{'cast_id': 6, '...|[{'credit_id': '5...| 4584|


cres的数据类型：

`DataFrame[cast: string, crew: string, id: int]`

* 也要对这个部分做一样的操作，取出导演和主演

In [19]:
transfer2 = udf(lambda x:eval(x), ArrayType(MapType(StringType(),StringType())))

In [20]:
getdirector = udf(lambda x:[i['name'] for i in x if i['job'] == 'Director'],StringType())

In [21]:
getcharacter = udf(lambda x:[i['name'] for i in x],StringType())

In [22]:
cres2 = cres.select('id',transfer2('cast').alias('cast'),transfer2('crew').alias('crew'))
cres2.dtypes

[('id', 'int'), ('cast', 'array<map<string,string>>'), ('crew', 'array<map<string,string>>')]

In [23]:
cres3 = cres2.select('id',getcharacter('cast').alias('Character'),getdirector('crew').alias('Director'))
cres3.show()

+-----+--------------------+--------------------+
|   id|           Character|            Director|
+-----+--------------------+--------------------+
|  862|[Tom Hanks, Tim A...|     [John Lasseter]|
| 8844|[Robin Williams, ...|      [Joe Johnston]|
|15602|[Walter Matthau, ...|     [Howard Deutch]|
|31357|[Whitney Houston,...|   [Forest Whitaker]|
|11862|[Steve Martin, Di...|     [Charles Shyer]|
|  949|[Al Pacino, Rober...|      [Michael Mann]|
|11860|[Harrison Ford, J...|    [Sydney Pollack]|
|45325|[Jonathan Taylor ...|      [Peter Hewitt]|
| 9091|[Jean-Claude Van ...|       [Peter Hyams]|
|  710|[Pierce Brosnan, ...|   [Martin Campbell]|
| 9087|[Michael Douglas,...|        [Rob Reiner]|
|12110|[Leslie Nielsen, ...|        [Mel Brooks]|
|21032|[Kevin Bacon, Bob...|       [Simon Wells]|
|10858|[Anthony Hopkins,...|      [Oliver Stone]|
| 1408|[Geena Davis, Mat...|      [Renny Harlin]|
|  524|[Robert De Niro, ...|   [Martin Scorsese]|
| 4584|[Kate Winslet, Em...|           [Ang Lee]|


_______________________________________________________________

In [24]:
kws = spark.read.csv('/Users/apple/Desktop/5003-project/moviedata/TMD/keywords.csv',multiLine=True,escape='"', header=True, inferSchema=True)

In [25]:
kws.show()

+-----+--------------------+
|   id|            keywords|
+-----+--------------------+
|  862|[{'id': 931, 'nam...|
| 8844|[{'id': 10090, 'n...|
|15602|[{'id': 1495, 'na...|
|31357|[{'id': 818, 'nam...|
|11862|[{'id': 1009, 'na...|
|  949|[{'id': 642, 'nam...|
|11860|[{'id': 90, 'name...|
|45325|                  []|
| 9091|[{'id': 949, 'nam...|
|  710|[{'id': 701, 'nam...|
| 9087|[{'id': 833, 'nam...|
|12110|[{'id': 3633, 'na...|
|21032|[{'id': 1994, 'na...|
|10858|[{'id': 840, 'nam...|
| 1408|[{'id': 911, 'nam...|
|  524|[{'id': 383, 'nam...|
| 4584|[{'id': 420, 'nam...|
|    5|[{'id': 612, 'nam...|
| 9273|[{'id': 409, 'nam...|
|11517|[{'id': 380, 'nam...|
+-----+--------------------+
only showing top 20 rows



In [27]:
getkws = udf(lambda x:[i['name'] for i in x],StringType())

In [28]:
kws= kws.select('id',getcharacter('keywords').alias('keywords'))
kws.show()

+-----+--------------------+
|   id|            keywords|
+-----+--------------------+
|  862|[jealousy, toy, b...|
| 8844|[board game, disa...|
|15602|[fishing, best fr...|
|31357|[based on novel, ...|
|11862|[baby, midlife cr...|
|  949|[robbery, detecti...|
|11860|[paris, brother b...|
|45325|                  []|
| 9091|[terrorist, hosta...|
|  710|[cuba, falsely ac...|
| 9087|[white house, usa...|
|12110|    [dracula, spoof]|
|21032|[wolf, dog-sleddi...|
|10858|[usa president, p...|
| 1408|[exotic island, t...|
|  524|[poker, drug abus...|
| 4584|[bowling, based o...|
|    5|[hotel, new year'...|
| 9273|[africa, indigeno...|
|11517|[brother brother ...|
+-----+--------------------+
only showing top 20 rows



kws的数据类型：

`DataFrame[id: int, keywords: string]`

In [26]:
kws = kws.select('id',transfer2('keywords').alias('keywords'))
kws.dtypes

[('id', 'int'), ('keywords', 'array<map<string,string>>')]

In [11]:
df2 = spark.read.csv('/Users/apple/Desktop/5003-project/moviedata/TMD/ratings.csv',multiLine=True,escape='"', header=True, inferSchema=True)

In [12]:
rats = df2.select(df2['userId'].alias('id'),'movieID','rating')
rats.show()

+---+-------+------+
| id|movieID|rating|
+---+-------+------+
|  1|    110|   1.0|
|  1|    147|   4.5|
|  1|    858|   5.0|
|  1|   1221|   5.0|
|  1|   1246|   5.0|
|  1|   1968|   4.0|
|  1|   2762|   4.5|
|  1|   2918|   5.0|
|  1|   2959|   4.0|
|  1|   4226|   4.0|
|  1|   4878|   5.0|
|  1|   5577|   5.0|
|  1|  33794|   4.0|
|  1|  54503|   3.5|
|  1|  58559|   4.0|
|  1|  59315|   5.0|
|  1|  68358|   5.0|
|  1|  69844|   5.0|
|  1|  73017|   5.0|
|  1|  81834|   5.0|
+---+-------+------+
only showing top 20 rows



In [13]:
rats.cache()

DataFrame[id: int, movieID: int, rating: double]

rats的数据类型：

`DataFrame[id: int, movieID: int, rating: double]`

### 到这个部分，所有的数据都读取进来，完成了清理，并转换成了需要的格式！

* 新的数据预处理部分

In [23]:
tomap = udf(lambda s: eval(s), ArrayType(MapType(StringType(),StringType())))

In [13]:
bmd2.show()

+--------------------+-----+--------------------+--------------------+------------+----------+
|              genres|   id|             tagline|               title|vote_average|vote_count|
+--------------------+-----+--------------------+--------------------+------------+----------+
|[{'id': 16, 'name...|  862|                null|           Toy Story|         7.7|      5415|
|[{'id': 12, 'name...| 8844|Roll the dice and...|             Jumanji|         6.9|      2413|
|[{'id': 10749, 'n...|15602|Still Yelling. St...|    Grumpier Old Men|         6.5|        92|
|[{'id': 35, 'name...|31357|Friends are the p...|   Waiting to Exhale|         6.1|        34|
|[{'id': 35, 'name...|11862|Just When His Wor...|Father of the Bri...|         5.7|       173|
|[{'id': 28, 'name...|  949|A Los Angeles Cri...|                Heat|         7.7|      1886|
|[{'id': 35, 'name...|11860|You are cordially...|             Sabrina|         6.2|       141|
|[{'id': 28, 'name...|45325|The Original Bad ...| 

In [30]:
bmd5 = bmd2.select(tomap('genres').alias('genres'),'id')

In [31]:
bmd5.show()

+--------------------+-----+
|              genres|   id|
+--------------------+-----+
|[Map(name -> Anim...|  862|
|[Map(name -> Adve...| 8844|
|[Map(name -> Roma...|15602|
|[Map(name -> Come...|31357|
|[Map(name -> Come...|11862|
|[Map(name -> Acti...|  949|
|[Map(name -> Come...|11860|
|[Map(name -> Acti...|45325|
|[Map(name -> Acti...| 9091|
|[Map(name -> Adve...|  710|
|[Map(name -> Come...| 9087|
|[Map(name -> Come...|12110|
|[Map(name -> Fami...|21032|
|[Map(name -> Hist...|10858|
|[Map(name -> Acti...| 1408|
|[Map(name -> Dram...|  524|
|[Map(name -> Dram...| 4584|
|[Map(name -> Crim...|    5|
|[Map(name -> Crim...| 9273|
|[Map(name -> Acti...|11517|
+--------------------+-----+
only showing top 20 rows



In [41]:
bmd5.dtypes

[('genres', 'array<map<string,string>>'), ('id', 'int')]

In [43]:
takewords = udf(lambda x: [i['name'] for i in x], StringType())

In [48]:
takewords = udf(lambda x: [i['name'] for i in x], StringType())
bmd6 = bmd5.select(takewords('genres').alias('genres'))

In [49]:
bmd6

DataFrame[genres: string]

In [50]:
bmd6.dtypes

[('genres', 'string')]

In [51]:
bmd6.

[Row(genres='[Animation, Comedy, Family]')]

In [180]:
import pandas as pd

In [187]:
type(md)

<class 'pandas.core.frame.DataFrame'>

In [153]:
print(bmd2['genres'][1])

Column<b'genres[1]'>


In [155]:
genres=bmd2.select('genres')
genres.show()

+--------------------+
|              genres|
+--------------------+
|[{'id': 16, 'name...|
|[{'id': 12, 'name...|
|[{'id': 10749, 'n...|
|[{'id': 35, 'name...|
|[{'id': 35, 'name...|
|[{'id': 28, 'name...|
|[{'id': 35, 'name...|
|[{'id': 28, 'name...|
|[{'id': 28, 'name...|
|[{'id': 12, 'name...|
|[{'id': 35, 'name...|
|[{'id': 35, 'name...|
|[{'id': 10751, 'n...|
|[{'id': 36, 'name...|
|[{'id': 28, 'name...|
|[{'id': 18, 'name...|
|[{'id': 18, 'name...|
|[{'id': 80, 'name...|
|[{'id': 80, 'name...|
|[{'id': 28, 'name...|
+--------------------+
only showing top 20 rows



In [168]:
t = list(genres['genres'])
t

Name: org.apache.toree.interpreter.broker.BrokerException
Message: Traceback (most recent call last):
  File "/var/folders/g4/bh0x216x6_j83zjyf806bjn40000gn/T/kernel-PySpark-8a4a6204-b0a8-4b06-8dd1-258e515fd5a6/pyspark_runner.py", line 180, in <module>
    eval(compiled_code)
  File "<string>", line 1, in <module>
  File "/usr/local/Cellar/apache-spark/2.2.1/libexec/python/pyspark/sql/column.py", line 250, in __iter__
    raise TypeError("Column is not iterable")
TypeError: Column is not iterable

StackTrace: org.apache.toree.interpreter.broker.BrokerState$$anonfun$markFailure$1.apply(BrokerState.scala:158)
org.apache.toree.interpreter.broker.BrokerState$$anonfun$markFailure$1.apply(BrokerState.scala:158)
scala.Option.foreach(Option.scala:257)
org.apache.toree.interpreter.broker.BrokerState.markFailure(BrokerState.scala:157)
sun.reflect.GeneratedMethodAccessor54.invoke(Unknown Source)
sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
java.lang.reflec

In [113]:
from ast import literal_eval

In [127]:
bmd4 = bmd2.withColumn('genres2',bmd2.genres)

In [133]:
bmd2

DataFrame[genres: string, id: int, tagline: string, title: string, vote_average: double, vote_count: int]

In [142]:
from pyspark.sql.types import StringType
from pyspark.sql.functions import udf

In [141]:
bmd3 = bmd2.select('id',bmd2['genres'].cast('ARRAY<MAP<string,string>>'),'tagline','title','vote_average','vote_count')

Name: org.apache.toree.interpreter.broker.BrokerException
Message: Traceback (most recent call last):
  File "/usr/local/Cellar/apache-spark/2.2.1/libexec/python/pyspark/sql/utils.py", line 63, in deco
    return f(*a, **kw)
  File "/usr/local/Cellar/apache-spark/2.2.1/libexec/python/lib/py4j-0.10.4-src.zip/py4j/protocol.py", line 319, in get_return_value
    format(target_id, ".", name), value)
py4j.protocol.Py4JJavaError: An error occurred while calling o684.select.
: org.apache.spark.sql.AnalysisException: cannot resolve '`genres`' due to data type mismatch: cannot cast StringType to ArrayType(MapType(StringType,StringType,true),true);;
'Project [id#222, unresolvedalias(cast(genres#3 as array<map<string,string>>), None), tagline#19, title#20, vote_average#22, vote_count#23]
+- Project [genres#3, cast(id#5 as int) AS id#222, tagline#19, title#20, vote_average#22, vote_count#23]
   +- Project [genres#3, id#5, tagline#19, title#20, vote_average#22, vote_count#23]
      +- Relation[adul

In [135]:
bmd4

DataFrame[genres: string, id: int, tagline: string, title: string, vote_average: double, vote_count: int, genres2: string]

In [93]:
type(bmd2.select(['genres']).asDict())

Name: org.apache.toree.interpreter.broker.BrokerException
Message: Traceback (most recent call last):
  File "/var/folders/g4/bh0x216x6_j83zjyf806bjn40000gn/T/kernel-PySpark-8a4a6204-b0a8-4b06-8dd1-258e515fd5a6/pyspark_runner.py", line 180, in <module>
    eval(compiled_code)
  File "<string>", line 1, in <module>
  File "/usr/local/Cellar/apache-spark/2.2.1/libexec/python/pyspark/sql/dataframe.py", line 1020, in __getattr__
    "'%s' object has no attribute '%s'" % (self.__class__.__name__, name))
AttributeError: 'DataFrame' object has no attribute 'asDict'

StackTrace: org.apache.toree.interpreter.broker.BrokerState$$anonfun$markFailure$1.apply(BrokerState.scala:158)
org.apache.toree.interpreter.broker.BrokerState$$anonfun$markFailure$1.apply(BrokerState.scala:158)
scala.Option.foreach(Option.scala:257)
org.apache.toree.interpreter.broker.BrokerState.markFailure(BrokerState.scala:157)
sun.reflect.GeneratedMethodAccessor54.invoke(Unknown Source)
sun.reflect.DelegatingMethodAccessorImp

In [79]:
bmd3 = bmd2.select('id',eval(bmd2['genres']),'tagline','title','vote_average','vote_count')

Name: org.apache.toree.interpreter.broker.BrokerException
Message: Traceback (most recent call last):
  File "/var/folders/g4/bh0x216x6_j83zjyf806bjn40000gn/T/kernel-PySpark-8a4a6204-b0a8-4b06-8dd1-258e515fd5a6/pyspark_runner.py", line 180, in <module>
    eval(compiled_code)
  File "<string>", line 1, in <module>
TypeError: eval() arg 1 must be a string, bytes or code object

StackTrace: org.apache.toree.interpreter.broker.BrokerState$$anonfun$markFailure$1.apply(BrokerState.scala:158)
org.apache.toree.interpreter.broker.BrokerState$$anonfun$markFailure$1.apply(BrokerState.scala:158)
scala.Option.foreach(Option.scala:257)
org.apache.toree.interpreter.broker.BrokerState.markFailure(BrokerState.scala:157)
sun.reflect.GeneratedMethodAccessor54.invoke(Unknown Source)
sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
java.lang.reflect.Method.invoke(Method.java:498)
py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
py4j.reflection.ReflectionEng

In [64]:
print(bmd2['genres'])

Column<b'genres'>


In [74]:
type(bmd2.select('genres'))

<class 'pyspark.sql.dataframe.DataFrame'>

In [76]:
type(bmd2.select('genres').collect())

<class 'list'>

In [78]:
bmd2.select('genres').collect()[0]

Row(genres="[{'id': 16, 'name': 'Animation'}, {'id': 35, 'name': 'Comedy'}, {'id': 10751, 'name': 'Family'}]")

In [ ]:
bmd2.select('genres').collect()

In [58]:
bmd3 = bmd2.select('id',bmd2['genres'].astype('MAP{'string','string'}'),'tagline','title','vote_average','vote_count')

Name: org.apache.toree.interpreter.broker.BrokerException
Message: Traceback (most recent call last):
  File "/var/folders/g4/bh0x216x6_j83zjyf806bjn40000gn/T/kernel-PySpark-8a4a6204-b0a8-4b06-8dd1-258e515fd5a6/pyspark_runner.py", line 174, in <module>
    ast_parsed = ast.parse(final_code)
  File "/usr/local/anaconda3/lib/python3.6/ast.py", line 35, in parse
    return compile(source, filename, mode, PyCF_ONLY_AST)
  File "<unknown>", line 1
    bmd3 = bmd2.select('id',bmd2['genres'].astype('MAP{'string','string'}'),'tagline','title','vote_average','vote_count')
                                                             ^
SyntaxError: invalid syntax

StackTrace: org.apache.toree.interpreter.broker.BrokerState$$anonfun$markFailure$1.apply(BrokerState.scala:158)
org.apache.toree.interpreter.broker.BrokerState$$anonfun$markFailure$1.apply(BrokerState.scala:158)
scala.Option.foreach(Option.scala:257)
org.apache.toree.interpreter.broker.BrokerState.markFailure(BrokerState.scala:157)
sun.

In [41]:
bmd3 = bmd2.select('id',bmd2['genres'].astype(('string','string')),'tagline','title','vote_average','vote_count')

Name: org.apache.toree.interpreter.broker.BrokerException
Message: Traceback (most recent call last):
  File "/var/folders/g4/bh0x216x6_j83zjyf806bjn40000gn/T/kernel-PySpark-8a4a6204-b0a8-4b06-8dd1-258e515fd5a6/pyspark_runner.py", line 180, in <module>
    eval(compiled_code)
  File "<string>", line 1, in <module>
  File "/usr/local/Cellar/apache-spark/2.2.1/libexec/python/pyspark/sql/column.py", line 422, in cast
    raise TypeError("unexpected type: %s" % type(dataType))
TypeError: unexpected type: <class 'tuple'>

StackTrace: org.apache.toree.interpreter.broker.BrokerState$$anonfun$markFailure$1.apply(BrokerState.scala:158)
org.apache.toree.interpreter.broker.BrokerState$$anonfun$markFailure$1.apply(BrokerState.scala:158)
scala.Option.foreach(Option.scala:257)
org.apache.toree.interpreter.broker.BrokerState.markFailure(BrokerState.scala:157)
sun.reflect.GeneratedMethodAccessor54.invoke(Unknown Source)
sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:

In [36]:
help(map)

Help on class map in module builtins:

class map(object)
 |  map(func, *iterables) --> map object
 |  
 |  Make an iterator that computes the function using arguments from
 |  each of the iterables.  Stops when the shortest iterable is exhausted.
 |  
 |  Methods defined here:
 |  
 |  __getattribute__(self, name, /)
 |      Return getattr(self, name).
 |  
 |  __iter__(self, /)
 |      Implement iter(self).
 |  
 |  __new__(*args, **kwargs) from builtins.type
 |      Create and return a new object.  See help(type) for accurate signature.
 |  
 |  __next__(self, /)
 |      Implement next(self).
 |  
 |  __reduce__(...)
 |      Return state information for pickling.



In [46]:
type(bmd2.select('genres'))

<class 'pyspark.sql.dataframe.DataFrame'>

In [78]:
type(bmd2.select('genres').toPandas())

<class 'pandas.core.frame.DataFrame'>

In [51]:
t = list(bmd2.select('genres'))

[Column<b'genres'>]

In [58]:
type(bmd2.select('genres').toPandas())

<class 'pandas.core.frame.DataFrame'>

In [59]:
bmd2.select('genres').toPandas().count()

genres    45466
dtype: int64

In [70]:
import pandas as pd
import numpy as np
from pandas import Series, DataFrame

In [76]:
bmd2.select('genres').toPandas()

                                                  genres
0      [{'id': 16, 'name': 'Animation'}, {'id': 35, '...
1      [{'id': 12, 'name': 'Adventure'}, {'id': 14, '...
2      [{'id': 10749, 'name': 'Romance'}, {'id': 35, ...
3      [{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...
4                         [{'id': 35, 'name': 'Comedy'}]
5      [{'id': 28, 'name': 'Action'}, {'id': 80, 'nam...
6      [{'id': 35, 'name': 'Comedy'}, {'id': 10749, '...
7      [{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...
8      [{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...
9      [{'id': 12, 'name': 'Adventure'}, {'id': 28, '...
10     [{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...
11     [{'id': 35, 'name': 'Comedy'}, {'id': 27, 'nam...
12     [{'id': 10751, 'name': 'Family'}, {'id': 16, '...
13     [{'id': 36, 'name': 'History'}, {'id': 18, 'na...
14     [{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...
15     [{'id': 18, 'name': 'Drama'}, {'id': 80, 'name...
16     [{'id': 18, 'name': 'Dra

In [31]:
type(bmd2[0])

<class 'pyspark.sql.column.Column'>

In [35]:
from ast import literal_eval

In [23]:
trdd = sc.parallelize()

Name: org.apache.toree.interpreter.broker.BrokerException
Message: Traceback (most recent call last):
  File "/var/folders/g4/bh0x216x6_j83zjyf806bjn40000gn/T/kernel-PySpark-50d89198-10f6-4b2b-a2c7-a0c9170742cf/pyspark_runner.py", line 180, in <module>
    eval(compiled_code)
  File "<string>", line 1, in <module>
  File "/usr/local/Cellar/apache-spark/2.2.1/libexec/python/pyspark/context.py", line 491, in parallelize
    serializer.dump_stream(c, tempFile)
  File "/usr/local/Cellar/apache-spark/2.2.1/libexec/python/pyspark/serializers.py", line 220, in dump_stream
    self.serializer.dump_stream(self._batched(iterator), stream)
  File "/usr/local/Cellar/apache-spark/2.2.1/libexec/python/pyspark/serializers.py", line 139, in dump_stream
    self._write_with_length(obj, stream)
  File "/usr/local/Cellar/apache-spark/2.2.1/libexec/python/pyspark/serializers.py", line 149, in _write_with_length
    serialized = self.dumps(obj)
  File "/usr/local/Cellar/apache-spark/2.2.1/libexec/python/py

In [10]:
temp = bmd2

In [11]:
bmd2 = temp

In [13]:
type(bmd2['genres'])

<class 'pyspark.sql.column.Column'>

In [24]:
type(bmd2.select('genres'))

<class 'pyspark.sql.dataframe.DataFrame'>

In [39]:
bmd2.select('genres').map(lambda x:[x])

Name: org.apache.toree.interpreter.broker.BrokerException
Message: Traceback (most recent call last):
  File "/var/folders/g4/bh0x216x6_j83zjyf806bjn40000gn/T/kernel-PySpark-b9c3eaff-bb2b-4924-8e47-d33dfce5670c/pyspark_runner.py", line 180, in <module>
    eval(compiled_code)
  File "<string>", line 1, in <module>
  File "/usr/local/Cellar/apache-spark/2.2.1/libexec/python/pyspark/sql/dataframe.py", line 1020, in __getattr__
    "'%s' object has no attribute '%s'" % (self.__class__.__name__, name))
AttributeError: 'DataFrame' object has no attribute 'map'

StackTrace: org.apache.toree.interpreter.broker.BrokerState$$anonfun$markFailure$1.apply(BrokerState.scala:158)
org.apache.toree.interpreter.broker.BrokerState$$anonfun$markFailure$1.apply(BrokerState.scala:158)
scala.Option.foreach(Option.scala:257)
org.apache.toree.interpreter.broker.BrokerState.markFailure(BrokerState.scala:157)
sun.reflect.GeneratedMethodAccessor42.invoke(Unknown Source)
sun.reflect.DelegatingMethodAccessorImpl.i

In [35]:
for i in bmd2.select('genres'):
    print(i)

Column<b'genres'>


In [12]:
bmd2['genres'] = bmd2.select('genres')

Name: org.apache.toree.interpreter.broker.BrokerException
Message: Traceback (most recent call last):
  File "/var/folders/g4/bh0x216x6_j83zjyf806bjn40000gn/T/kernel-PySpark-b9c3eaff-bb2b-4924-8e47-d33dfce5670c/pyspark_runner.py", line 180, in <module>
    eval(compiled_code)
  File "<string>", line 1, in <module>
TypeError: 'DataFrame' object does not support item assignment

StackTrace: org.apache.toree.interpreter.broker.BrokerState$$anonfun$markFailure$1.apply(BrokerState.scala:158)
org.apache.toree.interpreter.broker.BrokerState$$anonfun$markFailure$1.apply(BrokerState.scala:158)
scala.Option.foreach(Option.scala:257)
org.apache.toree.interpreter.broker.BrokerState.markFailure(BrokerState.scala:157)
sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
java.lang.reflect.Method.invoke(Method.java:498)
py4j.reflecti